In [1]:
#Fine-Tuning BERT with a Logistic Regression Layer


import torch
from torch import nn
from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
dataset = load_dataset("imdb")


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize the data
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [6]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Limit the dataset size
small_train_dataset = tokenized_datasets["train"].select(range(1000))  # Select first 1000 samples
small_eval_dataset = tokenized_datasets["test"].select(range(500))     # Select first 500 samples


In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Smaller batch size
    per_device_eval_batch_size=4,   # Smaller batch size for evaluation
    num_train_epochs=1,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)


/home/j/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer.train()


Epoch,Training Loss,Validation Loss



KeyboardInterrupt



In [ ]:
# Extract embeddings (logits) from BERT
def extract_embeddings(model, dataset):
    embeddings = []
    labels = []
    with torch.no_grad():
        for batch in dataset:
            inputs = {
                "input_ids": batch["input_ids"].to(model.device),
                "attention_mask": batch["attention_mask"].to(model.device)
            }
            outputs = model(**inputs)
            embeddings.append(outputs.logits.cpu().numpy())
            labels.append(batch["label"].cpu().numpy())
    return embeddings, labels

train_embeddings, train_labels = extract_embeddings(model, tokenized_datasets["train"])
test_embeddings, test_labels = extract_embeddings(model, tokenized_datasets["test"])

# Train a logistic regression model on top of the embeddings
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(train_embeddings, train_labels)

# Predict and evaluate
train_preds = log_reg.predict(train_embeddings)
test_preds = log_reg.predict(test_embeddings)

train_accuracy = accuracy_score(train_labels, train_preds)
test_accuracy = accuracy_score(test_labels, test_preds)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
